In [45]:
from newscatcher import Newscatcher
import pandas as pd
from datetime import datetime

In [46]:
all_headlines_csv = 'data/google-news-headlines.csv'
all_df_headlines = pd.read_csv(all_headlines_csv)

In [47]:
now = datetime.utcnow().strftime('%Y-%m-%d %H:%M')
now

'2020-02-27 19:57'

In [48]:
# get the current headlines
ns = Newscatcher('news.google.com')
df_latest = pd.DataFrame({'datetime': now, 'headline': ns.headlines})

In [49]:
df_latest

,datetime,headline
0,2020-02-27 19:57,The 31 wildest lines from Donald Trump's self-...
1,2020-02-27 19:57,Diagnosis Of Coronavirus Patient In California...
2,2020-02-27 19:57,South Korean Leader Said Coronavirus Would ‘Di...
3,2020-02-27 19:57,Shooter at Milwaukee Molson Coors had a long-r...
4,2020-02-27 19:57,Coronavirus case with potential unknown origin...
5,2020-02-27 19:57,Ex-Baltimore mayor sentenced to 3 years in pri...
6,2020-02-27 19:57,Nancy Pelosi calls stock market plunge 'distur...
7,2020-02-27 19:57,‘Doomsday cult’ mom Lori Vallow returns to Ida...
8,2020-02-27 19:57,Jimmy Kimmel mocks Pence's ability to tackle c...
9,2020-02-27 19:57,20-foot waves may be coming to the Great Lakes...


In [50]:
# merge with the old one
all_df_headlines = df_headlines.append(df_latest)


In [51]:
all_df_headlines.drop_duplicates(['headline'], inplace=True)

In [52]:
# save all headlines  to file
all_df_headlines.to_csv(all_headlines_csv)

In [ ]:
# TODO count those with coronavirus